In [1]:
!pip install langchain-community langchain-text-splitters  pypdf  langchain-huggingface sentence-transformers faiss-cpu PyMuPDF   --no-deps

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 51.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 329.1/329.1 kB 38.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 73.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 20.1 MB/s eta 0:00:00


In [2]:
# 第一個 Cell
!pip install -q bitsandbytes langchain-huggingface sentencepiece
!pip install -q -U transformers accelerate
# 注意：這裡刻意不寫 torch，直接用 Colab 內建的，就不會打架也不會慢

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.4/566.4 kB 21.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
transformers 5.0.0 requires huggingface-hub<2.0,>=1.3.0, but you have huggingface-hub 0.36.2 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.3/10.3 MB 55.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 553.3/553.3 kB 18.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-huggingface 1.2.0 requires huggingface-hub<1.0.0,>=0.33.4, but you have huggingface-hub 1.4.1 which is incompatible.


In [4]:
from transformers import Qwen2_5_VLForConditionalGeneration, AutoProcessor, pipeline
from langchain_huggingface import HuggingFacePipeline, ChatHuggingFace
import torch
# 改用 VLM 模型 (例如 Qwen2.5-VL-3B-Instruct)
model_id = "Qwen/Qwen2.5-VL-3B-Instruct"

# 1. 載入 VLM 專用的模型與 Processor
model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    device_map="auto",
    trust_remote_code=True
)

# VLM 必須使用 Processor (包含 Tokenizer 和 Image Processor)
processor = AutoProcessor.from_pretrained(model_id, trust_remote_code=True)

# 2. 建立 Pipeline (關鍵：task 設為 image-to-text)
# 注意：在 Transformers 4.49+ 中，建議指定正確的 task
vlm_pipeline = pipeline(
    "image-text-to-text",  # 必須全小寫
    model=model,
    processor=processor,   # 直接傳入 processor，它會自動處理文字與影像
    device_map="auto"
)
vlm_pipeline.model.generation_config.max_new_tokens = 512
vlm_pipeline.model.generation_config.temperature = 0.2
# 3. 轉為 LangChain 物件
llm = HuggingFacePipeline(pipeline=vlm_pipeline)
image_model = ChatHuggingFace(llm=llm)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

Loading weights:   0%|          | 0/824 [00:00<?, ?it/s]

...

The image processor of type `Qwen2VLImageProcessor` is now loaded as a fast processor by default, even if the model checkpoint was saved with a slow processor. This is a breaking change and may produce slightly different outputs. To continue using the slow processor, instantiate this class with `use_fast=False`. 


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

In [5]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, BitsAndBytesConfig
from langchain_huggingface import HuggingFacePipeline, ChatHuggingFace
model_id = "Qwen/Qwen3-4B-Instruct-2507"

# ==========================================
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    device_map="auto",
    trust_remote_code=True
)

# 確保 pipeline 正常運作的常見修復
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
# ==========================================
# 建立原生 Transformers Pipeline
text_generation_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=128,      # 決定回答長度
    temperature=0.2,         # RAG 建議設低一點，減少幻覺
    repetition_penalty=1.1,  # 避免鬼打牆
    return_full_text=False   # 重要：設為 False 只回傳生成的部分
)

# . 轉為 LangChain 的 LLM 物件
llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

# (關鍵) 轉為 ChatModel 物件
# ChatHuggingFace 會自動處理 prompt template (User/System/Assistant 格式)
chat_model = ChatHuggingFace(llm=llm)

config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

...

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

Loading weights:   0%|          | 0/398 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/238 [00:00<?, ?B/s]

Passing `generation_config` together with generation-related arguments=({'temperature', 'repetition_penalty', 'max_new_tokens'}) is deprecated and will be removed in future versions. Please pass either a `generation_config` object OR all generation parameters explicitly, but not both.


In [7]:
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
embedding_model = HuggingFaceEmbeddings(
    model_name="Qwen/Qwen3-Embedding-0.6B",
    model_kwargs={'device': 'cpu'},
)

Loading weights:   0%|          | 0/310 [00:00<?, ?it/s]

In [8]:
import fitz
from PIL import Image
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
import os
import io
import base64
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_community.vectorstores import FAISS
import faiss

In [9]:
path=r"/content/drive/MyDrive/RAG/RAG資料集/2401.15884v3.pdf"
chunk_size=512
chunk_overlap=64
query="What the CRAG proposed to improve"

In [10]:
text_data=[]
img_data=[]

讀取pdf+圖片

In [33]:
with fitz.open(path) as pdf_file:
    if not os.path.exists("extracted images"):
        os.makedirs("extracted images")

    for page_number in range(len(pdf_file)):
        page=pdf_file[page_number]

        text=page.get_text().strip()
        text_data.append({"respond":text, "name":page_number+1})

        images=page.get_images(full=True)

        for image_index, img in enumerate(images,start=0):
            xref=img[0]
            base_image=pdf_file.extract_image(xref)
            image_bytes=base_image["image"]
            image_ext=base_image["ext"]

            image=Image.open(io.BytesIO(image_bytes))
            image.save(f"extracted images/image_{page_number+1}_{image_index+1}.{image_ext}")

讓VLM看圖片後給SUMMARY

In [21]:
system_template = (
      "You are an assistant tasked with summarizing tables, images and text for retrieval. "
      "These summaries will be embedded and used to retrieve the raw text or table elements. "
      "Give a concise summary of the table or text that is well optimized for retrieval. "
      "Table or text or image:"
  )
prompt=ChatPromptTemplate.from_messages([
     ("system", system_template),
     ("user",[
        {"type":"text","text":"Here is the image/table/text to summarize:"},
        {"type":"image_url","image_url":{"url": "data:image/jpeg;base64,{b64_img}"}}
  ])
])

def get_image_data(inputs):
  image_path = inputs["image_path"]
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')
def clean_text(text):
    return text.strip()

chain = (
    RunnablePassthrough.assign(b64_img=get_image_data)
    | prompt
    | image_model
    | StrOutputParser()
    | RunnableLambda(clean_text)
)

image_folder = "extracted images"
all_files = sorted([f for f in os.listdir(image_folder) if f.endswith(('.png', '.jpg', '.jpeg'))])
batch_inputs = [{"image_path": os.path.join(image_folder, f)} for f in all_files]
batch_responses = chain.batch(batch_inputs)
for file_name, summary in zip(all_files, batch_responses):
    img_data.append({
        "response": summary,  # 模型的摘要結果
        "name": file_name     # 圖片檔名
    })


Both `max_new_tokens` (=256) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=256) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=256) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=256) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


分解與創建vectorstore

In [22]:
docs_list=[Document(page_content=text["respond"],metadata={"name":text["name"]}) for text in text_data]
image_list=[Document(page_content=image["response"],metadata={"name":image["name"]}) for image in img_data]

text_splitter=RecursiveCharacterTextSplitter(chunk_size=chunk_size,chunk_overlap=chunk_overlap)

doc_splits = text_splitter.split_documents(docs_list)
img_splits = text_splitter.split_documents(image_list)

In [25]:
from tqdm import tqdm
from langchain_community.vectorstores import FAISS

# 1. 合併所有待處理的文檔
all_docs = doc_splits + img_splits

# 2. 先用第一筆資料初始化 vectorstore (這很快)
vectorstore = FAISS.from_documents([all_docs[0]], embedding_model)

# 3. 剩下的資料用迴圈加入，並加上進度條
# batch_size 建議設為 4-8，兼顧進度更新頻率與 CPU 運算效率
batch_size = 5
with tqdm(total=len(all_docs), desc="CPU Embedding Processing") as pbar:
    pbar.update(1) # 更新第一筆初始化的進度
    for i in range(1, len(all_docs), batch_size):
        batch = all_docs[i : i + batch_size]
        vectorstore.add_documents(batch)
        pbar.update(len(batch))

# 4. 建立 retriever
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={'k': 3},
)

CPU Embedding Processing: 100%|██████████| 152/152 [34:03<00:00, 13.44s/it]


In [26]:
docs=retriever.invoke(query)

In [32]:
print(docs)

[Document(id='9a6b4766-abd5-4d17-b31d-62bd9f396ab2', metadata={'name': 2}, page_content='Generation (CRAG) is proposed to self-correct\nthe results of retriever and improve the utilization\nof documents for augmenting generation.\nA\nlightweight retrieval evaluator is designed to\nassess the overall quality of retrieved documents\nfor a query. This serves as a crucial component\nin RAG, contributing to informative generation\nby\nreviewing\nand\nevaluating\nthe\nrelevance\nand reliability of the retrieved documents.\nA\nconfidence degree is quantified based on which'), Document(id='54e77864-e149-4da4-9e27-a95e87734ff8', metadata={'name': 15}, page_content='improvement of our method.\nSelf-CRAG: To demonstrate that our plug-and-\nplay approach can be utilized in other concurrent\nstudies, we specifically designed to insert our\nCRAG into the Self-RAG (Asai et al., 2024)\nframework and named it Self-CRAG. Self-RAG'), Document(id='74d637b1-148b-4bbc-9e61-c7530176b6b7', metadata={'name': 5

In [27]:
system = """You are an assistant for question-answering tasks. Answer the question based upon your knowledge.
Use three-to-five sentences maximum and keep the answer concise."""
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "Retrieved documents: \n\n <docs>{documents}</docs> \n\n User question: <question>{question}</question>"),
    ]
)

rag_chain=prompt | chat_model | StrOutputParser()

texts = "\n".join([doc.page_content for doc in docs])

generation = rag_chain.invoke({"documents":text, "question": query})

Both `max_new_tokens` (=128) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


In [31]:
print(generation)

CRAG proposes to improve retrieval and generation by introducing a critic model that decides whether to retrieve and which retrieved document to use for generation. It enhances decision-making by evaluating retrieved content before using it in the generation process. This helps in selecting appropriate information based on context—such as correct, incorrect, or ambiguous cases—to improve overall accuracy.
